# **EXTRACCIÓN DE TWEETS**

In [ ]:
# Instalamos la libreria SNscrape para la extración de los tweets e importamos algunas librerias para la manipulación de datos
!pip3 install snscrape
import snscrape.modules.twitter as twitter_scraper
import csv

In [ ]:
# Definimos los criterios de búsqueda
R_TOTAL = 10000
search_texts_array = [
    "sputnik",
    "sinopharm",
    "pfizer",
    "johnson & johnson",
    "astrazeneca",
    "vacuna china",
    "vacuna rusa",
    "vacuna covid",
    "desconfianza vacuna",
    "antivacuna",
    "efecto vacuna"
]
  
coordinates_array = [
    {'country': 'La Paz', 'latitude': -16.4942, 'longitude': -68.1475, 'radius': '50km'},
    {'country': 'Cochabamba', 'latitude': -17.3935, 'longitude': -66.157, 'radius': '50km'},
    {'country': 'Cobija', 'latitude': -11.0183, 'longitude': -68.7537, 'radius': '50km'},
    {'country': 'Trinidad', 'latitude': -14.8333, 'longitude': -64.9, 'radius': '50km'},
    {'country': 'Oruro', 'latitude': -17.9667, 'longitude': -67.1167, 'radius': '50km'},
    {'country': 'Potosi', 'latitude': -19.5833, 'longitude': -65.75, 'radius': '50km'},
    {'country': 'Santa Cruz', 'latitude': -17.7892, 'longitude': -63.1975, 'radius': '50km'},
    {'country': 'Tarija', 'latitude': -21.5317, 'longitude': -64.7311, 'radius': '50km'},
    {'country': 'Sucre', 'latitude': -19.0431, 'longitude': -65.2592, 'radius': '50km'}
]

In [ ]:
def search_by_location(search_texts, coordinates):
    print("Initialize extraction processing...")
    count = 0
    index = 0
    while index < len(coordinates):
        tweets = []
        csv_file = open(coordinates[index]['country'].replace(" ", "") + ".csv", "a", encoding="utf-8")
        csv_writer = csv.writer(csv_file)
        for index2, search in enumerate(search_texts):
            scraper = twitter_scraper.TwitterSearchScraper(
                search + ' since:2020-11-01 until:2021-10-31 lang:es geocode:' + str(coordinates[index]['latitude']) + ',' +
                str(coordinates[index]['longitude']) + ',' + coordinates[index]['radius'])
            for i, tweet in enumerate(scraper.get_items()):
                if i > TOTAL:
                    break
                save_tweet(csv_writer, tweet)
                count += 1                                    

        csv_file.close()
        index = index + 1
    print("FINISHED! With {} records".format(count))


def search_all(search_texts):
    print("Initialize extraction processing...")
    count = 0
    csv_file = open("vacunas_bolivia.csv", "a", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    for index, search in enumerate(search_texts):
        scraper = twitter_scraper.TwitterSearchScraper(search + ' since:2020-11-01 until:2021-10-31 lang:es')
        for i, tweet in enumerate(scraper.get_items()):
            if i > TOTAL:
              break
            if "Bolivia" in tweet.user.location:
              save_tweet(csv_writer, tweet)

            count += 1
    csv_file.close()
    print("SCRIPT FINISHED!! With {} records".format(count))


def save_tweet(csv_writer, tweet):
    plc_name = tweet.place.name if (tweet.place is not None) else ""
    plc_country = tweet.place.country if (tweet.place is not None) else ""
    plc_country_code = tweet.place.countryCode if (tweet.place is not None) else ""
    coord_latitude = tweet.coordinates.latitude if (tweet.coordinates is not None) else ""
    coord_longitude = tweet.coordinates.longitude if (tweet.coordinates is not None) else ""
    csv_writer.writerow([
        tweet.id,
        tweet.date,
        tweet.user.id,
        tweet.user.username,
        tweet.user.displayname,
        tweet.user.created,
        tweet.user.followersCount,
        tweet.user.location,
        coord_latitude,
        coord_longitude,
        plc_name,
        plc_country,
        plc_country_code,
        tweet.lang,
        tweet.conversationId,
        tweet.content,
        tweet.url,
        tweet.replyCount,
        tweet.retweetedTweet,
        tweet.retweetCount,
        tweet.likeCount,
        tweet.hashtags,
        tweet.cashtags,
        tweet.media,
        tweet.sourceLabel
])
    

search_by_location(search_texts_array, coordinates_array)

search_all(search_texts_array)

In [ ]:
import pandas as pd

data_lpz = pd.read_csv("LaPaz.csv", header=None, encoding = "utf-8")
data_cba = pd.read_csv("Cochabamba.csv", header=None, encoding = "utf-8")
data_pnd = pd.read_csv("Cobija.csv", header=None, encoding = "utf-8")
data_tja = pd.read_csv("Tarija.csv", header=None, encoding = "utf-8")
data_ben = pd.read_csv("Trinidad.csv", header=None, encoding = "utf-8")
data_oru = pd.read_csv("Oruro.csv", header=None, encoding = "utf-8")
data_pts = pd.read_csv("Potosi.csv", header=None, encoding = "utf-8")
data_scz = pd.read_csv("SantaCruz.csv", header=None, encoding = "utf-8")
data_chq = pd.read_csv("Sucre.csv", header=None, encoding = "utf-8")
data_all = pd.read_csv("vacunas_bolivia.csv", header=None, encoding = "utf-8")

frames = [data_lpz, data_cba, data_pnd, data_tja, data_ben, data_oru, data_pts, data_scz, data_chq, data_all]
data_project = pd.concat(frames)

data_project.to_csv('rawDataset1.csv', header=False, index=False)

In [ ]:
# Definimos una funcion para buscar las respuestas y comentarios a un tweet
def search_comments():
    count = 0
    df = pd.read_csv('rawDataset1.csv', header=None, encoding = "utf-8")
    col_names = ['id', 'date', 'user_id', 'user_username', 'user_displayname', 'user_created',
                 'user_followers_count', 'user_location', 'coordinates_latitude', 'coordinates_longitude',
                 'place_name', 'place_country', 'place_country_code', 'language', 'conversation_id', 'content',
                 'url', 'reply_count', 'retweeted_tweet', 'retweet_count', 'like_count', 'hashtags', 'cashtags',
                 'media', 'source']
    df.columns = col_names
    df = df.drop_duplicates(df.columns[~df.columns.isin(['id'])], keep='first')
    print(df.shape)
    csv_file = open("rawDataset2.csv", "a", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    for i in df.index:
        scraper = twitter_scraper.TwitterTweetScraper(tweetId=df['id'][i], mode=twitter_scraper.TwitterTweetScraperMode.RECURSE)
        for j, tweet in enumerate(scraper.get_items()):
            save_tweet(csv_writer, tweet)
            count += 1
            print(tweet.id)
    print('{} records saved!!!'.format(count))

search_comments()